# Clustergrammer heatmap

#### This notebooks generate the json files needed to plot the [Clustergrammer](https://clustergrammer.readthedocs.io/) heatmaps.

Author: [Daniel Domingo-Fernández](https://github.com/ddomingof) 

Running time: about 5 minutes

In [1]:
import time
import sys
import os

from clustergrammer import Network
import pandas as pd
from collections import defaultdict
import itertools as itt

In [2]:
time.asctime()

'Wed Feb 21 14:32:57 2018'

In [3]:
print(sys.version)

3.4.5 (default, Dec 11 2017, 14:22:24) 
[GCC 4.8.5 20150623 (Red Hat 4.8.5-16)]


In [4]:
%matplotlib inline

Define location of gene set files via relative paths

In [5]:
#Location of the cloned repo
BASE_PATH = os.environ['COMPATH']

In [6]:
kegg_excel = os.path.join(BASE_PATH,'src','compath','static','resources','excel','kegg_gene_sets.csv')
reactome_excel= os.path.join(BASE_PATH,'src','compath','static','resources','excel','reactome_gene_sets.csv')
wikipathways_excel = os.path.join(BASE_PATH,'src','compath','static','resources','excel','wikipathways_gene_sets.csv')

In [7]:
def create_pathway_gene_set_dict(dataframe):
    """Creates a pathway genes dictionary
    
    :param pandas.DataFrame dataset: gene sets df
    :rtype: collections.defaultdict
    :returns: dictionary of pathway gene sets
    """
    
    pathway_dictionary = defaultdict(set)
    
    for pathway_name in dataframe: # iterate over columns in dataframe

        for gene in dataframe[pathway_name].unique():
            if not isinstance(gene, str): # There are NaN in the Pandas nArray
                continue

            pathway_dictionary[pathway_name].add(gene)
            
    return pathway_dictionary

Load KEGG

In [8]:
kegg_dataframe = pd.read_csv(kegg_excel, dtype=object)

# Remove the 'Homo sapiens' out of the KEGG pathways
kegg_dataframe.columns = [
    kegg_pathway.replace(' - Homo sapiens (human)', '')
    for kegg_pathway in kegg_dataframe
] 

kegg_pathways = create_pathway_gene_set_dict(kegg_dataframe)
        
assert (len(kegg_pathways.keys()) == 323)

Load Reactome

In [9]:
reactome_dataframe = pd.read_csv(reactome_excel, dtype=object)

reactome_pathways = create_pathway_gene_set_dict(reactome_dataframe)

assert (len(reactome_pathways.keys()) == 2132) # Total of 2636 of those: 2132 are not empty

Load WikiPathways

In [10]:
wikipathways_dataframe = pd.read_csv(wikipathways_excel, dtype=object)

wikipathways_pathways = create_pathway_gene_set_dict(wikipathways_dataframe)

assert (len(wikipathways_pathways.keys()) == 408)

In [11]:
def create_similarity_matrix(dataset):
    """Creates a similarity matrix for a given pathway-geneset dataset
    
    :param dict dataset: pathway gene set dictionary
    :rtype: pandas.DataFrame
    :returns: similarity matrix
    """
    
    index = sorted(dataset.keys())
    similarity_dataframe = pd.DataFrame(0.0, index=index, columns=index)
    
    for pathway_1, pathway_2 in itt.product(index, index):

        intersection = len(dataset[pathway_1].intersection(dataset[pathway_2]))
        smaller_set = min(len(dataset[pathway_1]), len(dataset[pathway_2]))
                
        similarity = float(intersection/smaller_set) # Formula to calculate similarity
            
        similarity_dataframe[pathway_1][pathway_2] = similarity
        
    return similarity_dataframe

Create and visualize similarity matrices

In [13]:
kegg_similarity_matirx = create_similarity_matrix(kegg_pathways)
reactome_similarity_matirx = create_similarity_matrix(reactome_pathways) # Comment this line in case it takes too much time running
wikipathways_similarity_matirx = create_similarity_matrix(wikipathways_pathways)

In [14]:
kegg_similarity_matirx

,2-Oxocarboxylic acid metabolism,ABC transporters,AGE-RAGE signaling pathway in diabetic complications,AMPK signaling pathway,Acute myeloid leukemia,Adherens junction,Adipocytokine signaling pathway,Adrenergic signaling in cardiomyocytes,African trypanosomiasis,"Alanine, aspartate and glutamate metabolism",...,Vitamin B6 metabolism,Vitamin digestion and absorption,Wnt signaling pathway,alpha-Linolenic acid metabolism,beta-Alanine metabolism,cAMP signaling pathway,cGMP-PKG signaling pathway,mRNA surveillance pathway,mTOR signaling pathway,p53 signaling pathway
2-Oxocarboxylic acid metabolism,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.222222,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ABC transporters,0.000000,1.000000,0.000000,0.022727,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.041667,0.000000,0.000000,0.000000,0.045455,0.000000,0.000000,0.000000,0.000000
AGE-RAGE signaling pathway in diabetic complications,0.000000,0.000000,1.000000,0.111111,0.318182,0.125000,0.159420,0.161616,0.352941,0.000000,...,0.000000,0.000000,0.151515,0.000000,0.000000,0.202020,0.131313,0.000000,0.171717,0.073529
AMPK signaling pathway,0.000000,0.022727,0.111111,1.000000,0.227273,0.041667,0.478261,0.225000,0.000000,0.000000,...,0.000000,0.000000,0.016667,0.000000,0.032258,0.150000,0.133333,0.175824,0.266667,0.044118
Acute myeloid leukemia,0.000000,0.000000,0.318182,0.227273,1.000000,0.090909,0.151515,0.075758,0.000000,0.000000,...,0.000000,0.000000,0.106061,0.000000,0.000000,0.272727,0.136364,0.000000,0.409091,0.015152
Adherens junction,0.000000,0.000000,0.125000,0.041667,0.090909,1.000000,0.000000,0.027778,0.000000,0.000000,...,0.000000,0.000000,0.263889,0.000000,0.000000,0.125000,0.055556,0.000000,0.069444,0.000000
Adipocytokine signaling pathway,0.000000,0.000000,0.159420,0.478261,0.151515,0.000000,1.000000,0.043478,0.029412,0.000000,...,0.000000,0.000000,0.043478,0.000000,0.000000,0.159420,0.086957,0.000000,0.173913,0.000000
Adrenergic signaling in cardiomyocytes,0.000000,0.000000,0.161616,0.225000,0.075758,0.027778,0.043478,1.000000,0.176471,0.000000,...,0.000000,0.000000,0.084507,0.000000,0.000000,0.472222,0.500000,0.208791,0.041667,0.000000
African trypanosomiasis,0.000000,0.000000,0.352941,0.000000,0.000000,0.000000,0.029412,0.176471,1.000000,0.000000,...,0.000000,0.041667,0.205882,0.000000,0.000000,0.000000,0.147059,0.000000,0.117647,0.029412
"Alanine, aspartate and glutamate metabolism",0.222222,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,...,0.000000,0.041667,0.000000,0.000000,0.096774,0.000000,0.000000,0.000000,0.000000,0.000000


In [15]:
reactome_similarity_matirx

,2-LTR circle formation,5-Phosphoribose 1-diphosphate biosynthesis,A tetrasaccharide linker sequence is required for GAG synthesis,ABC transporter disorders,ABC transporters in lipid homeostasis,ABC-family proteins mediated transport,ADP signalling through P2Y purinoceptor 1,ADP signalling through P2Y purinoceptor 12,AKT phosphorylates targets in the cytosol,AKT phosphorylates targets in the nucleus,...,t(4;14) translocations of FGFR3,tRNA Aminoacylation,tRNA modification in the mitochondrion,tRNA modification in the nucleus and cytosol,tRNA processing,tRNA processing in the mitochondrion,tRNA processing in the nucleus,trans-Golgi Network Vesicle Budding,truncated APC mutants destabilize the destruction complex,vRNP Assembly
2-LTR circle formation,1.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.00000,0.00,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0
5-Phosphoribose 1-diphosphate biosynthesis,0.0,1.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.00000,0.00,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0
A tetrasaccharide linker sequence is required for GAG synthesis,0.0,0.0,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.00000,0.00,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0
ABC transporter disorders,0.0,0.0,0.0,1.000000,0.277778,0.901235,0.000000,0.000000,0.000000,0.000000,...,0.0,0.00000,0.00,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0
ABC transporters in lipid homeostasis,0.0,0.0,0.0,0.277778,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.00000,0.00,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0
ABC-family proteins mediated transport,0.0,0.0,0.0,0.901235,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.00000,0.00,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0
ADP signalling through P2Y purinoceptor 1,0.0,0.0,0.0,0.000000,0.000000,0.000000,1.000000,0.772727,0.000000,0.000000,...,0.0,0.00000,0.00,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0
ADP signalling through P2Y purinoceptor 12,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.772727,1.000000,0.000000,0.000000,...,0.0,0.00000,0.00,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0
AKT phosphorylates targets in the cytosol,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.333333,...,0.0,0.00000,0.00,0.000000,0.000000,0.0,0.000000,0.0,0.071429,0.0
AKT phosphorylates targets in the nucleus,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,1.000000,...,0.0,0.00000,0.00,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0


In [16]:
wikipathways_similarity_matirx

,Hfe effect on hepcidin production,"4-hydroxytamoxifen, Dexamethasone, and Retinoic Acids Regulation of p27 Expression",ACE Inhibitor Pathway,AGE/RAGE pathway,AMP-activated Protein Kinase (AMPK) Signaling,ATM Signaling Network in Development and Disease,ATM Signaling Pathway,ATR Signaling,Acetylcholine Synthesis,Adipogenesis,...,miRNA Biogenesis,miRNA Regulation of DNA Damage Response,miRNA regulation of p53 pathway in prostate cancer,miRNA regulation of prostate cancer signaling pathways,miRNA targets in ECM and membrane receptors,miRNAs involved in DNA damage response,miRs in Muscle Cell Differentiation,mir-124 predicted interactions with cell cycle and differentiation,mir34a and TGIF2 in osteoclastogenesis,p38 MAPK Signaling Pathway
Hfe effect on hepcidin production,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
"4-hydroxytamoxifen, Dexamethasone, and Retinoic Acids Regulation of p27 Expression",0.000000,1.000000,0.000000,0.277778,0.277778,0.166667,0.000000,0.000000,0.000000,0.000000,...,0.000,0.000000,0.000000,0.277778,0.000000,0.000000,0.000000,0.142857,0.0,0.166667
ACE Inhibitor Pathway,0.000000,0.000000,1.000000,0.058824,0.000000,0.000000,0.000000,0.000000,0.000000,0.117647,...,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
AGE/RAGE pathway,0.000000,0.277778,0.058824,1.000000,0.030303,0.043478,0.100000,0.000000,0.000000,0.151515,...,0.000,0.060606,0.088235,0.155556,0.000000,0.000000,0.111111,0.000000,0.0,0.205882
AMP-activated Protein Kinase (AMPK) Signaling,0.000000,0.277778,0.000000,0.030303,1.000000,0.086957,0.075000,0.000000,0.000000,0.102941,...,0.000,0.044118,0.029412,0.088889,0.000000,0.040000,0.055556,0.428571,0.0,0.000000
ATM Signaling Network in Development and Disease,0.000000,0.166667,0.000000,0.043478,0.086957,1.000000,0.325000,0.222222,0.000000,0.000000,...,0.000,0.260870,0.058824,0.066667,0.000000,0.043478,0.000000,0.285714,0.0,0.147059
ATM Signaling Pathway,0.000000,0.000000,0.000000,0.100000,0.075000,0.325000,1.000000,0.222222,0.000000,0.075000,...,0.000,0.675000,0.147059,0.125000,0.000000,0.200000,0.000000,0.000000,0.0,0.088235
ATR Signaling,0.000000,0.000000,0.000000,0.000000,0.000000,0.222222,0.222222,1.000000,0.000000,0.000000,...,0.000,0.555556,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
Acetylcholine Synthesis,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,...,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
Adipogenesis,0.000000,0.000000,0.117647,0.151515,0.102941,0.000000,0.075000,0.000000,0.000000,1.000000,...,0.000,0.075269,0.029412,0.066667,0.000000,0.060000,0.083333,0.000000,0.0,0.117647


Export to Clustergrammer-JS

In [ ]:
net = Network()
net.load_df(kegg_similarity_matirx)

# # Z-score normalize the rows
# net.normalize(axis='row', norm_type='zscore', keep_orig=True)

# # filter for the top 100 columns based on their absolute value sum
# net.filter_N_top('col', 100, 'sum')

net.cluster()

In [15]:
# save visualization JSON to file for use by front end
net.write_json_to_file('viz', 'kegg_clustergrammer.json')

In [19]:
net = Network()
net.load_df(reactome_similarity_matirx)

# filter for the top 300 columns/rows based on their absolute value sum
net.filter_N_top('col', 300, 'sum')
net.filter_N_top('row', 300, 'sum')
net.cluster()
net.write_json_to_file('viz', 'reactome_clustergrammer.json')

In [24]:
net = Network()
net.load_df(wikipathways_similarity_matirx)

net.cluster()
net.write_json_to_file('viz', 'wikipathways_clustergrammer.json')